In [1]:
import osmnx as ox, pandas as pd, networkx as nx, time, geopandas as gpd, os, json
%matplotlib inline
ox.config(use_cache=True, log_file=True, log_console=True, log_filename='get_every_us_city_graph',
          data_folder='G:/Geoff/osmnx/cities-usa', cache_folder='G:/Geoff/osmnx/cache/cities-usa')

In [2]:
network_type = 'drive_service'
retain_all = True
buffer = False

In [3]:
input_folder = 'input_data/places' #tigerfile place shapefiles
output_folder = 'G:/Geoff/osmnx/cities-usa' #where to save graph shapefiles and graphml

In [4]:
with open('input_data/states_by_fips.json') as f:
    fips_to_state = json.load(f)

In [5]:
def buffer_geometry(cities):
    buffer_dist = 50
    cities.name = 'cities'
    cities_utm = ox.project_gdf(cities)
    cities_utm['geometry'] = cities_utm['geometry'].buffer(buffer_dist)
    cities_buffered = ox.project_gdf(cities_utm, to_latlong=True)
    print('Buffered geometry.')
    return(cities_buffered)

## Run it

In [6]:
# for each state shapefile folder in the folder of state shapefile folders
for state_shapefile_folder in os.listdir(input_folder):
    
    # load the shapefile for this state
    start_time = time.time()
    gdf = gpd.read_file('{}/{}'.format(input_folder, state_shapefile_folder))
    
    # create the output path to save to
    state_fips = gdf['STATEFP'].unique()[0]
    state_folder = '{}_{}'.format(state_fips, fips_to_state[state_fips]['abbreviation'])
    output_path = '{}/{}'.format(output_folder, state_folder)
    
    # filter geometries by non-CDP, except if this is hawaii, don't, because they're all CDPs there
    if state_fips == '15':
        cities = gdf
    else:
        cities = gdf[~gdf['NAMELSAD'].str.contains('CDP')]
    print('{} loaded {} rows, {} cities'.format(state_folder, len(gdf), len(cities)))
          
    # tigerline data is epsg:4269, but osm uses epsg:4326, so project it
    cities = cities.to_crs({'init':'epsg:4326'})
    cities = cities.sort_values(by='GEOID', ascending=True)
    
    # optionally buffer so streets that form the boundary don't get ignored
    # but, buffering will mess up using ALAND for area as polygon area will capture nodes outside ALAND!
    if buffer:
        cities = buffer_geometry(cities)
        
    # create list of queries
    cities['city_folder'] = cities.apply(lambda row: '{}_{}'.format(row['GEOID'], row['NAME']).replace(' ', '_'), axis=1)
    queries = cities.apply(lambda row: {'city_folder':row['city_folder'], 'polygon':row['geometry']}, axis=1).tolist()
    
    count = 0
    for query in queries:
        try:
            # load graph and save it if it hasn't already been saved in the output_path
            if not os.path.exists('{}/{}.graphml'.format(output_path, query['city_folder'])):
                graph_name = '{}_{}'.format(state_folder, query['city_folder'])
                G = ox.graph_from_polygon(polygon=query['polygon'], network_type=network_type, 
                                          name=graph_name, retain_all=retain_all) 
                ox.save_graph_shapefile(G, folder=output_path, filename=query['city_folder'])
                ox.save_graphml(G, folder=output_path, filename='{}.graphml'.format(query['city_folder']))
            count += 1
        except Exception as e:
            print('"{}" failed: {}'.format(query['city_folder'], e))

    print('Finished making {} {} graphs in {:,.2f} seconds\n'.format(count, state_folder, time.time()-start_time))

01_AL loaded 585 rows, 461 cities
Finished making 461 01_AL graphs in 2,224.72 seconds

02_AK loaded 355 rows, 148 cities
"0201090_Akutan" failed: There are no data elements in the response JSON objects
"0212680_Chefornak" failed: There are no data elements in the response JSON objects
"0219060_Diomede" failed: There are no data elements in the response JSON objects
"0227640_Gambell" failed: There are no data elements in the response JSON objects
"0242160_Kupreanof" failed: No geometry data set yet (expected in column 'geometry'.
Finished making 143 02_AK graphs in 382.36 seconds

04_AZ loaded 451 rows, 91 cities
Finished making 91 04_AZ graphs in 1,246.36 seconds

05_AR loaded 541 rows, 502 cities
Finished making 502 05_AR graphs in 1,461.04 seconds

06_CA loaded 1522 rows, 482 cities
Finished making 482 06_CA graphs in 5,811.56 seconds

08_CO loaded 458 rows, 271 cities
Finished making 271 08_CO graphs in 1,705.14 seconds

09_CT loaded 143 rows, 30 cities
Finished making 30 09_CT gra

Self-intersection at or near point -82.722619999999992 27.983628999999997


"1212875_Clearwater" failed: Shape does not have a valid geometry
"1267650_South_Palm_Beach" failed: 'node'
Finished making 408 12_FL graphs in 3,219.28 seconds

13_GA loaded 625 rows, 538 cities
"1379164_Vernonburg" failed: No geometry data set yet (expected in column 'geometry'.
Finished making 537 13_GA graphs in 3,074.90 seconds

15_HI loaded 151 rows, 151 cities
"1506325_Eden_Roc" failed: 'node'
"1507542_Fern_Acres" failed: No geometry data set yet (expected in column 'geometry'.
"1507675_Fern_Forest" failed: No geometry data set yet (expected in column 'geometry'.
Finished making 148 15_HI graphs in 462.16 seconds

16_ID loaded 227 rows, 201 cities
Finished making 201 16_ID graphs in 705.37 seconds

17_IL loaded 1368 rows, 1297 cities
"1737803_Irwin" failed: No geometry data set yet (expected in column 'geometry'.
Finished making 1296 17_IL graphs in 6,941.95 seconds

18_IN loaded 679 rows, 567 cities
Finished making 567 18_IN graphs in 2,825.29 seconds

19_IA loaded 1008 rows, 9